In [ ]:
# install osmnx through pip, conda version is too old
# osmnx version should be 1.2.2 and shapely 2.0.0

# example plot using distance from a point (edges are not contained)
import osmnx as ox
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

bbox = ox.utils_geo.bbox_from_point((47.377126, 8.5465143), dist=500)
poly = ox.utils_geo.bbox_to_poly(*bbox)

buildings = ox.geometries_from_polygon(poly, tags={'building': True})
roads = ox.graph_from_polygon(poly)
nodes, edges = ox.graph_to_gdfs(roads)
# G = ox.graph_from_point((47.6710676, 9.0544013), dist=1000, network_type='all')

buildings.plot(ax=ax, facecolor='black', alpha=1)
edges.plot(ax=ax, linewidth=1, edgecolor='red')

In [ ]:
# downloading from swisstopo using a GPS bounding box

import swisstopo_helpers as sw
from pathlib import Path

# from pyproj import Transformer

# # from gps to lv95
# trans = Transformer.from_crs(4326, 2056)

def transform_osm_to_sw(north, south, east, west):
  # [LONGITUDE_WEST, LAT_SOUTH, LONG_EAST, LAT_NORTH]
  return [west, south, east, north]

class Arguments(object):
  date_range = None
  bbox = None
  resolution = 0.1
  save_dir = Path("./swisstopo").resolve().absolute()

args = Arguments()

#use bounding box from above
args.bbox = transform_osm_to_sw(*bbox)
print(args.bbox)
url = sw.get_url(args)
print(url)
sw.download_tifs(url, args)


In [ ]:
# ploting the segmentation map from a swissimage tiff file with api
import re
import json
from pathlib import Path

for entry in args.save_dir.iterdir():
  if(entry.name.find("swissimage-dop10") == 0 and entry.name.endswith(".tif")):
    # todo, maybe skip if there are multiple time variants of this image?
    print(entry.name)
    image_id = re.search("swissimage-dop10_([0-9]+)_[0-9]+-[0-9]+", entry.name)[0]
    json_file = image_id + ".json"
    print(json_file)
    fileObject = open(args.save_dir.joinpath(json_file), "r")
    jsonContent = fileObject.read()
    coordList = json.loads(jsonContent)
    print(coordList)


    fig, ax = plt.subplots(figsize=(10, 10))

    # using the bounding box of the api TODO
    
    # might want to add some margin to also draw streets that go out of the image
    bbox = [coordList[1][1], coordList[0][1], coordList[0][0], coordList[3][0]]
    poly = ox.utils_geo.bbox_to_poly(*bbox)
    print(poly)
    # poly = ox.utils_geo.bbox_to_poly(47.3684871, 47.3776072, 8.5375049, 8.5509303)

    # load osm infos TODO tags
    buildings = ox.geometries_from_polygon(poly, tags={'building': True})
    roadnetwork = ox.graph_from_polygon(poly)
    nodes, edges = ox.graph_to_gdfs(roadnetwork)

    # plot osm data TODO road width
    buildings.plot(ax=ax, facecolor='black', alpha=1)
    edges.plot(ax=ax, linewidth=1, edgecolor='red')

    # corner dots
    ax.scatter([x for (x,y) in coordList], [y for (x,y) in coordList], s=40, c="green", alpha=0.5)
    ax.scatter([x for (x,y) in coordList], [y for (x,y) in coordList], s=5, c="red", alpha=1)

    # crop
    ax.set_xlim((bbox[2], bbox[3]))
    ax.set_ylim((bbox[0], bbox[1]))
    ax.set_axis_off()
    fig.savefig(args.save_dir.joinpath(f'{image_id}_segmentation.png'),bbox_inches='tight',pad_inches = 0,dpi=300)
